# Comparison

In [1]:
import os
from collections import OrderedDict

import numpy as np
import pandas as pd

import dendropy
from dendropy.calculate import popgenstat

## Genes

OrderedDict(): https://www.geeksforgeeks.org/ordereddict-in-python/

In [24]:
genes_species = OrderedDict()
my_species = ['RESTV', 'SUDV']
my_genes = ['NP', 'L', 'VP35', 'VP40']


for name in my_genes:
    gene_name = name.split('.')[0] # There is no '.' in gene names. Maybe this is for the future
    char_mat = dendropy.DnaCharacterMatrix.get_from_path('%s_align.fasta' % name, 'fasta')
    genes_species[gene_name] = {}
    
    for species in my_species:
        genes_species[gene_name][species] = dendropy.DnaCharacterMatrix()
    for taxon, char_map in char_mat.items():
        species = taxon.label.split('_')[0]
        if species in my_species:
            genes_species[gene_name][species].taxon_namespace.add_taxon(taxon)
            genes_species[gene_name][species][taxon] = char_map

In [25]:
genes_species

OrderedDict([('NP',
              {'RESTV': <dendropy.datamodel.charmatrixmodel.DnaCharacterMatrix at 0x2e05436a7c0>,
               'SUDV': <dendropy.datamodel.charmatrixmodel.DnaCharacterMatrix at 0x2e05436a8b0>}),
             ('L',
              {'RESTV': <dendropy.datamodel.charmatrixmodel.DnaCharacterMatrix at 0x2e05439fcd0>,
               'SUDV': <dendropy.datamodel.charmatrixmodel.DnaCharacterMatrix at 0x2e05436a160>}),
             ('VP35',
              {'RESTV': <dendropy.datamodel.charmatrixmodel.DnaCharacterMatrix at 0x2e054368a00>,
               'SUDV': <dendropy.datamodel.charmatrixmodel.DnaCharacterMatrix at 0x2e05436a520>}),
             ('VP40',
              {'RESTV': <dendropy.datamodel.charmatrixmodel.DnaCharacterMatrix at 0x2e0546c2c10>,
               'SUDV': <dendropy.datamodel.charmatrixmodel.DnaCharacterMatrix at 0x2e054603580>})])

We get four genes that we stored in the first recipe and aligned in the second. We load all the files (which are FASTA formatted) and create a dictionary with all of the genes. Each entry will be a dictionary itself with the RESTV or SUDV species, including all reads. This is not a lot of data, just a handful of genes.

numpy.ndarray: https://numpy.org/doc/stable/reference/generated/numpy.ndarray.html

In [3]:
np.ndarray(shape=(2,2), dtype=int, order='F')

array([[ 1139902158, -1676025184],
       [ 1088283143,  -887033182]])

popgenstat: https://dendropy.org/library/popgenstat.html

In [37]:
summary = np.ndarray(shape=(len(genes_species), 4 * len(my_species)))
stats = ['seg_sites', 'nuc_div', 'taj_d', 'wat_theta']
for row, (gene, species_data) in enumerate(genes_species.items()):
    for col_base, species in enumerate(my_species):
        summary[row, col_base * 4] = popgenstat.num_segregating_sites(species_data[species])
        summary[row, col_base * 4 + 1] = popgenstat.nucleotide_diversity(species_data[species])
        summary[row, col_base * 4 + 2] = popgenstat.tajimas_d(species_data[species])
        summary[row, col_base * 4 + 3] = popgenstat.wattersons_theta(species_data[species])
columns = []
for species in my_species:
    columns.extend(['%s (%s)' % (stat, species) for stat in stats])
df = pd.DataFrame(summary, index=genes_species.keys(), columns=columns)
df # vs print(df)

,seg_sites (RESTV),nuc_div (RESTV),taj_d (RESTV),wat_theta (RESTV),seg_sites (SUDV),nuc_div (SUDV),taj_d (SUDV),wat_theta (SUDV)
NP,113.0,0.020659,-0.482275,49.489051,118.0,0.029630,1.203522,56.64
L,288.0,0.018143,-0.295386,126.131387,282.0,0.024193,1.412350,135.36
VP35,43.0,0.017427,-0.553739,18.832117,50.0,0.027761,1.069061,24.00
VP40,61.0,0.026155,-0.188135,26.715328,41.0,0.023517,1.269160,19.68


I used a pandas DataFrame to print the results because it’s really tailored to deal with an
operation like this. We will initialize our DataFrame with a NumPy multidimensional array
with four rows (genes) and four statistics times the two species.
The statistics, such as the number of segregating sites, nucleotide diversity, Tajima’s D, and
Watterson’s theta, are computed by DendroPy. Note the placement of individual data points
in the array (the coordinate computation).
Look at the very last line: if you are in Jupyter, just putting df at the end will render the
DataFrame and the cell output, too. If you are not in a notebook, use print(df) (you can
also perform this in a notebook, but it will not look as pretty).

## Genomes

In [66]:
def do_basic_popgen(seqs):
    num_seg_sites = popgenstat.num_segregating_sites(seqs)
    avg_pair = popgenstat.average_number_of_pairwise_differences(seqs)
    nuc_div = popgenstat.nucleotide_diversity(seqs)
    print('Segregating sites: %d, Avg pairwise diffs: %.2f, Nucleotide diversity %.6f' % (num_seg_sites, avg_pair, nuc_div))
    print("Watterson's theta: %s" % popgenstat.wattersons_theta(seqs))
    print("Tajima's D: %s" % popgenstat.tajimas_d(seqs))

In [67]:
ebov_seqs = dendropy.DnaCharacterMatrix.get_from_path(
    'trim.fasta', schema='fasta', data_type='dna')
sl_2014 = []
drc_2007 = []
ebov2007_set = dendropy.DnaCharacterMatrix()
ebov2014_set = dendropy.DnaCharacterMatrix()
for taxon, char_map in ebov_seqs.items():
    print(taxon.label)
    if taxon.label.startswith('EBOV_2014') and len(sl_2014) < 8:
        sl_2014.append(char_map)
        ebov2014_set.taxon_namespace.add_taxon(taxon)
        ebov2014_set[taxon] = char_map
    elif taxon.label.startswith('EBOV_2007'):
        drc_2007.append(char_map)
        ebov2007_set.taxon_namespace.add_taxon(taxon)
        ebov2007_set[taxon] = char_map
        #ebov2007_set.extend_map({taxon: char_map})
del ebov_seqs

# +
print('2007 outbreak:')
print('Number of individuals: %s' % len(ebov2007_set.taxon_namespace))
do_basic_popgen(ebov2007_set)

print('\n2014 outbreak:')
print('Number of individuals: %s' % len(ebov2014_set.taxon_namespace))
do_basic_popgen(ebov2014_set)
# -

print(len(sl_2014))
print(len(drc_2007))



BDBV_KC545393 18728 bp
BDBV_KC545395 18728 bp
BDBV_KC545394 18728 bp
BDBV_KC545396 18728 bp
BDBV_FJ217161 18728 bp
TAFV_FJ217162 18728 bp
EBOV_2014_KM034549 18728 bp
EBOV_2014_KM034550 18728 bp
EBOV_2014_KM034554 18728 bp
EBOV_2014_KM034555 18728 bp
EBOV_2014_KM034556 18728 bp
EBOV_2014_KM034557 18728 bp
EBOV_2014_KM034560 18728 bp
EBOV_2014_KM034551 18728 bp
EBOV_2014_KM034552 18728 bp
EBOV_2014_KM034553 18728 bp
EBOV_2014_KM034558 18728 bp
EBOV_2014_KM034559 18728 bp
EBOV_2014_KM034561 18728 bp
EBOV_2014_KM034562 18728 bp
EBOV_2014_KM034563 18728 bp
EBOV_1976_AF272001 18728 bp
EBOV_1976_KC242801 18728 bp
EBOV_1995_KC242796 18728 bp
EBOV_1995_KC242799 18728 bp
EBOV_2007_KC242784 18728 bp
EBOV_2007_KC242785 18728 bp
EBOV_2007_KC242787 18728 bp
EBOV_2007_KC242786 18728 bp
EBOV_2007_KC242789 18728 bp
EBOV_2007_KC242788 18728 bp
EBOV_2007_KC242790 18728 bp
RESTV_AB050936 18728 bp
RESTV_JX477166 18728 bp
RESTV_FJ621585 18728 bp
RESTV_JX477165 18728 bp
RESTV_FJ621583 18728 bp
RESTV_FJ621584

In [68]:
pair_stats = popgenstat.PopulationPairSummaryStatistics(sl_2014, drc_2007)

print('Average number of pairwise differences irrespective of population: %.2f' %
      pair_stats.average_number_of_pairwise_differences)
print('Average number of pairwise differences between populations: %.2f' %
      pair_stats.average_number_of_pairwise_differences_between)
print('Average number of pairwise differences within populations: %.2f' %
      pair_stats.average_number_of_pairwise_differences_within)
print('Average number of net pairwise differences : %.2f' %
      pair_stats.average_number_of_pairwise_differences_net)
print('Number of segregating sites: %d' %
      pair_stats.num_segregating_sites)
print("Watterson's theta: %.2f" %
      pair_stats.wattersons_theta)
print("Wakeley's Psi: %.3f" % pair_stats.wakeleys_psi)
print("Tajima's D: %.2f" % pair_stats.tajimas_d)

Average number of pairwise differences irrespective of population: 284.46
Average number of pairwise differences between populations: 535.82
Average number of pairwise differences within populations: 10.50
Average number of net pairwise differences : 525.32
Number of segregating sites: 549
Watterson's theta: 168.84
Wakeley's Psi: 0.308
Tajima's D: 3.05
